In [1]:
#!pip3 install --upgrade sklearn 
#!pip3 uninstall xgboost


In [2]:
#The various imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
import pandas as pd

In [4]:
import xgboost as xgb

In [5]:
cancelled_flights = pd.read_csv("data/all_flight_data_spark.csv")

In [6]:
cancelled_flights = cancelled_flights.dropna()

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import TransformerMixin
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
#from xgboost import XGBClassifier

In [8]:
X = cancelled_flights[['OP_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST','CRS_DEP_TIME','CRS_ELAPSED_TIME','DISTANCE','WEEK','HOUR']]

In [9]:
y = cancelled_flights[['CANCELLED']]

In [10]:
X.dtypes

OP_CARRIER            object
OP_CARRIER_FL_NUM    float64
ORIGIN                object
DEST                  object
CRS_DEP_TIME         float64
CRS_ELAPSED_TIME     float64
DISTANCE             float64
WEEK                   int64
HOUR                   int64
dtype: object

In [11]:
# X['uniquecarrier'] = pd.Categorical(X['uniquecarrier'])
# X['flightnum'] = pd.Categorical(X['flightnum'])
# X['origin'] = pd.Categorical(X['origin'])
# X['dest'] = pd.Categorical(X['dest'])

In [12]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
categorical_cols = ['OP_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST']

ct = ColumnTransformer(
    [('le', OneHotEncoder(), categorical_cols)],
    remainder='passthrough'
)

In [14]:
X_trans = ct.fit_transform(X)

In [15]:
#X_trans = ct.transform(X) 


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_trans, y, random_state=42)

In [17]:
xgbclf = xgb.XGBClassifier() #LogisticRegressionCV(cv=cv, solver=solver, max_iter=max_iter)

pipe = Pipeline([('scaler', StandardScaler(with_mean=False)),
                 ('xgbclf', xgbclf)])

# The magical model.fit()
pipe.fit(X_train, y_train)

#xgbclf.fit(X_train, y_train)







/home/cdsw/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[12:31:56] WARNING: ../src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


Pipeline(steps=[('scaler', StandardScaler(with_mean=False)),
                ('xgbclf',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='approx',
                               validate_parameters=1, verbosity=None))])

In [18]:
train_score = pipe.score(X_train, y_train)
test_score = pipe.score(X_test, y_test)
print("train", train_score)
print("test", test_score)
#print(classification_report(y_test, pipe.predict(X_test)))
#data[labels.name + ' probability'] = pipe.predict_proba(X)[:, 1]

train 0.6868413699894086
test 0.6850156141307465


In [19]:
from joblib import dump, load

In [20]:
dump(pipe,'pipe.joblib')

['pipe.joblib']

In [21]:
dump(ct,'ct.joblib')

['ct.joblib']